In [1]:
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 62.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 76.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 118.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 28.2 MB/s eta 0:00:00


In [1]:
import glob
import os
from multiprocessing import Pool
from pathlib import Path
import pickle

import h5py
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from hdmf_zarr import NWBZarrIO

import utils.new_preprocess as nwp
import utils.nwb_dict_utils as nwb_utils

In [2]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
pd.options.mode.copy_on_write = True

In [3]:
def RMSE(x,y):
    return np.sqrt(np.mean((x-y)**2, -1))

def bias(x,y):
    return np.mean(x-y, -1)

def std_of_diff(x,y):
    return np.std(x-y, -1)

In [4]:
parents = sorted(glob.glob("/data/photometry/vary*"))
parents

['/data/photometry/vary_a_max',
 '/data/photometry/vary_a_power',
 '/data/photometry/vary_attenuation',
 '/data/photometry/vary_b_bright',
 '/data/photometry/vary_b_fast',
 '/data/photometry/vary_b_inf',
 '/data/photometry/vary_b_slow',
 '/data/photometry/vary_corr_s',
 '/data/photometry/vary_decay',
 '/data/photometry/vary_motion_power',
 '/data/photometry/vary_noise_std',
 '/data/photometry/vary_t_bright',
 '/data/photometry/vary_t_fast',
 '/data/photometry/vary_t_slow']

In [5]:
def evaluate(parent):
    perf = []
    params = []
    vary = parent.split('/')[-1][5:]
    for nwb_file in sorted(glob.glob(f"{parent}/*/*.nwb")):
        print(f"Processing NWB file: {nwb_file}")
        p = pd.read_csv(Path(nwb_file).parent / "parameters.csv", delimiter=",", index_col=0)
        p = list(p[vary])
        params.append([p[:3], p[3:6]])
        with h5py.File(Path(nwb_file).parent / 'groundtruth.h5') as file:
            dFF_gt = [file[f'fiber{f}']['dff'][:] for f in (0,1)]

        with NWBZarrIO(path=nwb_file, mode="r") as io:
            nwb = io.read()
            # convert nwb to dataframe
            df_from_nwb = nwb_utils.nwb_to_dataframe(nwb)
            # add the session column
            filename = Path(nwb_file).name
            session_name = filename.split(".")[0].split("FIP_")[1]
            df_from_nwb.insert(0, "session", session_name)
            # now pass the dataframe through the preprocessing function:
            df_fip_pp_nwb, df_PP_params = nwp.batch_processing_new(df_fip=df_from_nwb)
            # calculate performance measures
            tmp = []
            for pre in ("poly", "exp", "bright"):
                df = df_fip_pp_nwb[df_fip_pp_nwb["preprocess"]==pre]
                dFF = np.array([[df[(df["channel"] == ch) & (df["fiber_number"] == fiber)]["signal"]
                                 for ch in ['Iso', 'G', 'R']] for fiber in df_fip_pp_nwb["fiber_number"].unique()])
                tmp.append([RMSE(dFF, dFF_gt), bias(dFF, dFF_gt), std_of_diff(dFF, dFF_gt)])
            perf.append(tmp)

    return (np.array(params), # indices: expId, fiber, channel
            np.array(perf))   # indices: expId, method, performance_measure, fiber, channel

In [ ]:
%%time
# %%capture
results = Pool(int(os.environ['CO_CPUS'])).map(evaluate, parents)

Processing NWB file: /data/photometry/vary_a_max/FIP_000140_2024-08-06_10-01-07/FIP_000140_2024-08-06_10-01-07.nwb
Processing NWB file: /data/photometry/vary_a_power/FIP_000156_2024-08-06_10-01-19/FIP_000156_2024-08-06_10-01-19.nwb
Processing NWB file: /data/photometry/vary_corr_s/FIP_000164_2024-08-06_10-01-25/FIP_000164_2024-08-06_10-01-25.nwb
Processing NWB file: /data/photometry/vary_b_inf/FIP_000000_2024-08-06_09-59-21/FIP_000000_2024-08-06_09-59-21.nwbProcessing NWB file: /data/photometry/vary_b_bright/FIP_000042_2024-08-06_09-59-53/FIP_000042_2024-08-06_09-59-53.nwbProcessing NWB file: /data/photometry/vary_attenuation/FIP_000112_2024-08-06_10-00-46/FIP_000112_2024-08-06_10-00-46.nwbProcessing NWB file: /data/photometry/vary_b_fast/FIP_000028_2024-08-06_09-59-42/FIP_000028_2024-08-06_09-59-42.nwbProcessing NWB file: /data/photometry/vary_b_slow/FIP_000014_2024-08-06_09-59-32/FIP_000014_2024-08-06_09-59-32.nwb






/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in exp
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in multiply
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in add
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in exp
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in multiply
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in add
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in exp
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/caps

Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 19 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 18 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged 

/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in exp
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in add
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in multiply
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in exp
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in multiply
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in add
  return a * np.exp(-b * x) + c * np.exp(-d * x)


Processing NWB file: /data/photometry/vary_corr_s/FIP_000165_2024-08-06_10-01-26/FIP_000165_2024-08-06_10-01-26.nwb


...

Converged after 2 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Processing NWB file: /data/photometry/vary_t_fast/FIP_000071_2024-08-06_10-00-15/FIP_000071_2024-08-06_10-00-15.nwb
Converged after 20 iterations
Processing NWB file: /data/photometry/vary_motion_power/FIP_000133_2024-08-06_10-01-02/FIP_000133_2024-08-06_10-01-02.nwb
Converged after 20 iterations
Converged after 20 iterations


/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in exp
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in multiply
  return a * np.exp(-b * x) + c * np.exp(-d * x)
/root/capsule/code/utils/new_preprocess.py:96: RuntimeWarning: overflow encountered in add
  return a * np.exp(-b * x) + c * np.exp(-d * x)


Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations
Converged after 20 iterations


/opt/conda/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:503: RuntimeWarning: invalid value encountered in subtract
  dx = ((x0 + h) - x0)
/opt/conda/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:58: RuntimeWarning: invalid value encountered in subtract
  upper_dist = ub - x0
/opt/conda/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:591: RuntimeWarning: invalid value encountered in scalar subtract
  dx = x1[i] - x0[i]  # Recompute dx as exactly representable number.


Converged after 2 iterations
Converged after 20 iterations
Converged after 20 iterations


In [ ]:
# Save to a pickle file
with open('comparison5.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
rows = len(results)
fig = plt.figure(layout="constrained", figsize=(12,2.5*rows))
subfigs = fig.subfigures(rows, 3, wspace=0.05)
for row in range(rows):
    params, perf = results[row]
    vary = parents[row].split('/')[-1][5:]
    for col in range(3):
        ax0, ax1 = subfigs[row, col].subplots(1, 2, width_ratios=[4, 1], sharey=True)
        for k in range(len(perf[0])):
            ax0.scatter(params[...,0].ravel(), perf[:,k,col,:,0].ravel(), c=f'C{k}', s=50, marker='+')
            ax0.scatter(params[...,1].ravel(), perf[:,k,col,:,1].ravel(), c=f'C{k}', s=20, marker='o')
            ax0.scatter(params[...,2].ravel(), perf[:,k,col,:,2].ravel(), c=f'C{k}', s=50, marker='x')
            ax1.boxplot(perf[:,k,col].ravel(),
                        positions=[k], widths=.7, showmeans=True, notch=True,
                        boxprops=dict(color=f'C{k}'), flierprops=dict(markeredgecolor=f'C{k}'),
                        meanprops=dict(markerfacecolor=f'C{k}', markeredgecolor=f'C{k}', markersize=3))
            ax1.set_xticks([])
        if col+row==0:
            ax0.legend(handles=[Patch(color='C0', label='poly'),
                                Patch(color='C1', label='exp'),
                                Patch(color='C2', label='bright'),
                                Line2D([], [], color='k', markersize=7, marker='+', linestyle='None', label='Isosbestic'),
                                Line2D([], [], color='k', markersize=4, marker='o', linestyle='None', label='Green'),
                                Line2D([], [], color='k', markersize=7, marker='x', linestyle='None', label='Red')],
                      loc=(-.2,.4))
        ax0.set_xlabel(vary)
        ax0.set_ylabel(['RMSE', 'bias', 'std'][col])
        if row==0:
            ax0.set_title('dF/F')

In [10]:
# RMSE for each method over all simulations
print(np.mean([[results[row][1][:,m,0].mean() for m in (0,1,2)] for row in range(rows)], 0))
print(np.sum([[results[row][1][:,m,0].sum() for m in (0,1,2)] for row in range(rows)], 0))

[0.01165849 0.01565475 0.00903013]
[12.06533871 16.42508929  9.32073588]


In [ ]:
# HuberT(0.25)
[0.01165849 0.01565475 0.00903013]
[12.06533871 16.42508929  9.32073588]
# median values for all
[0.01165849 0.01565475 0.009519  ]
[12.06533871 16.42508929  9.81912743]
# median values with optim t_fast
[0.01165849 0.01565475 0.00961004]
[12.06533871 16.42508929  9.90852152]
# old
[0.01165849 0.01565475 0.00972095]
[12.06533871 16.42508929 10.03289963]

In [25]:
for parent in parents:
    vary = parent.split('/')[-1][5:]
    qq=[np.array(pd.read_csv(Path(nwb_file).parent / "parameters.csv", delimiter=",", index_col=0)[vary])
          for nwb_file in sorted(glob.glob(f"{parent}/*/*.nwb"))]
    print(vary, np.nanmedian(qq))

a_max 0.0559529356349044
a_power 0.9676361102731483
attenuation 0.990376054752377
b_bright 0.2588575845115088
b_fast 0.20126156478658955
b_inf 1124.1582304257604
b_slow 0.37137221696565875
corr_s 0.14418202106487135
decay 0.9531218852674423
motion_power 2.086924324699142
noise_std 0.0038102636392726
t_bright 2073.122248162862
t_fast 220.42119205670485
t_slow 3712.1658291263648
